<a href="https://colab.research.google.com/github/BarbaraAngelesOrtiz/ireland-energy-mix/blob/main/Energy_Ireland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL: Energy analysis in Ireland

In [ ]:
import pandas as pd
import os
import re
import shutil
from google.colab import files

url = "https://docs.google.com/spreadsheets/d/1uXgVtBHUrlrbxLVRYsmUlJwYt4fTyZ_X/export?format=xlsx"


output_folder = "output_netzero_one_row"
os.makedirs(output_folder, exist_ok=True)


energy_map = {
    # Fossil fuels
    "coal": "coal_peat",
    "bituminous_coal": "coal_peat",
    "anthracite_manufactured_ovoids": "coal_peat",
    "coke": "coal_peat",
    "lignite_brown_coal_briquettes": "coal_peat",
    "peat": "coal_peat",
    "milled_peat": "coal_peat",
    "sod_peat": "coal_peat",
    "briquettes": "coal_peat",

    "oil": "oil_petroleum",
    "crude": "oil_petroleum",
    "refinery_feedstocks": "oil_petroleum",
    "refinery_gas": "oil_petroleum",
    "gasoline": "oil_petroleum",
    "kerosene": "oil_petroleum",
    "jet_kerosene": "oil_petroleum",
    "fueloil": "oil_petroleum",
    "lpg": "oil_petroleum",
    "gasoil_diesel_derv": "oil_petroleum",
    "petroleum_coke": "oil_petroleum",
    "naphta": "oil_petroleum",
    "bitumen": "oil_petroleum",

    "natural_gas": "natural_gas",

    # Renewables
    "hydro": "renewables_hydro",
    "wind": "renewables_wind",
    "biomass": "renewables_bio",
    "renewable_waste": "renewables_bio",
    "landfill_gas": "renewables_bio",
    "biogas": "renewables_bio",
    "biodiesel": "renewables_bio",
    "bioethanol": "renewables_bio",
    "solar_photovoltaic": "renewables_solar",
    "solar_thermal": "renewables_solar",
    "ambient_heat": "other_renewables",

    # Others
    "nonrenewable_waste": "other_fossil",
    "electricity": "electricity",
    "heat": "heat",

    # Mantener explícito
    "total": "total"
}


indicators_keep = [
    "indigenous_production",
    "imports",
    "exports",
    "primary_energy_supply_incl_nonenergy",
    "primary_energy_requirement_excl_nonenergy",
    "transformation_input",
    "transformation_output",
    "own_use_and_distribution_losses",
    "available_final_energy_consumption",
    "total_final_energy_consumption",
    "industry",
    "transport",
    "residential",
    "commercialpublic_services",
    "agricultural"
]


def clean_label(s: str) -> str:
    """Normalizes labels for use as column names."""
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "", s)
    s = re.sub(r"_+", "_", s)
    return s


sheets = pd.read_excel(url, sheet_name=None, dtype=str)

rows_per_year = []

for i, (sheet_name, df) in enumerate(sheets.items(), start=1):
    # Detect year
    match_year = re.search(r"(19|20)\d{2}", str(sheet_name))
    year = int(match_year.group(0)) if match_year else None

    # Cleaning
    df = df.dropna(how="all").dropna(axis=1, how="all")
    df = df.rename(columns={df.columns[0]: "indicator"})
    df.columns = [clean_label(c) for c in df.columns]
    df["indicator"] = df["indicator"].apply(clean_label)

   # Filter only relevant indicators
    df = df[df["indicator"].isin(indicators_keep)]

    # Convert numeric
    for col in df.columns:
        if col != "indicator":
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

    # Group columns into energy categories
    grouped = {}
    for col, cat in energy_map.items():
        if col in df.columns:
            temp = df[["indicator", col]].copy()
            temp["category"] = cat
            for _, row in temp.iterrows():
                cname = f"{row['category']}__{row['indicator']}"
                grouped[cname] = grouped.get(cname, 0) + row[col]

    # Create a single row for this year
    row = {"year": year}
    row.update(grouped)
    rows_per_year.append(row)


final_df = pd.DataFrame(rows_per_year)

# Reorder columns
cols = ["year"] + sorted([c for c in final_df.columns if c != "year"])
final_df = final_df[cols]

# Delete columns with all zeros
final_df = final_df.loc[:, (final_df != 0).any(axis=0)]

# Save consolidated
final_path = os.path.join(output_folder, "energy_netzero_one_row_per_year.csv")
final_df.to_csv(final_path, index=False, encoding="utf-8-sig")

# ZIP and download
zip_path = shutil.make_archive("netzero_exports", "zip", output_folder)
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>